<a href="https://colab.research.google.com/github/ccstevie/nhl_model/blob/main/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import date

In [3]:
urlH = "https://www.naturalstattrick.com/games.php?fromseason=20212022&thruseason=20232024&stype=2&sit=all&loc=H&team=All&rate=n"
reqH = requests.get(urlH)
reqH.status_code

200

In [4]:
urlA = "https://www.naturalstattrick.com/games.php?fromseason=20212022&thruseason=20232024&stype=2&sit=all&loc=A&team=All&rate=n"
reqA = requests.get(urlA)
reqA.status_code

200

In [5]:
dfH = pd.read_html(urlH, header=0, na_values=["-"])[0]
dfH.drop(columns=["Unnamed: 2", "Game", "TOI", "Attendance", "GF%", "PDO", "SH%", "SV%"], inplace=True)
# dfH = dfH.iloc[:, : 29]
dfH = dfH.add_prefix('Home ')

In [6]:
dfH.head(10)

,Home Team,Home CF,Home CA,Home CF%,Home FF,Home FA,Home FF%,Home SF,Home SA,Home SF%,...,Home LDCA,Home LDCF%,Home LDSF,Home LDSA,Home LDSF%,Home LDGF,Home LDGA,Home LDGF%,Home LDSH%,Home LDSV%
0,Tampa Bay Lightning,55,58,48.67,40,47,45.98,28,35,44.44,...,31,55.71,12,9,57.14,0,1,0.0,0.00,88.89
1,Vegas Golden Knights,58,63,47.93,47,44,51.65,30,31,49.18,...,30,45.45,15,15,50.00,0,1,0.0,0.00,93.33
2,Toronto Maple Leafs,64,55,53.78,51,41,55.43,30,32,48.39,...,23,46.51,8,11,42.11,0,0,NaN,0.00,100.00
3,Washington Capitals,48,53,47.52,40,38,51.28,27,24,52.94,...,23,41.03,9,10,47.37,1,0,100.0,11.11,100.00
4,Colorado Avalanche,70,61,53.44,49,44,52.69,36,34,51.43,...,33,37.74,13,19,40.63,1,1,50.0,7.69,94.74
5,Edmonton Oilers,62,61,50.41,45,44,50.56,34,38,47.22,...,30,41.18,10,16,38.46,0,2,0.0,0.00,87.50
6,Anaheim Ducks,39,67,36.79,33,48,40.74,22,34,39.29,...,32,33.33,6,14,30.00,0,1,0.0,0.00,92.86
7,Buffalo Sabres,58,54,51.79,49,38,56.32,36,31,53.73,...,33,50.00,17,19,47.22,2,0,100.0,11.76,100.00
8,Ottawa Senators,68,75,47.55,53,63,45.69,36,48,42.86,...,26,38.10,10,16,38.46,0,1,0.0,0.00,93.75
9,Detroit Red Wings,45,69,39.47,35,57,38.04,26,48,35.14,...,32,41.82,8,13,38.10,0,0,NaN,0.00,100.00


In [7]:
dfA = pd.read_html(urlA, header=0, na_values=["-"])[0]
dfA.drop(columns=["Unnamed: 2", "Game", "TOI", "Attendance", "GF%", "PDO", "SH%", "SV%"], inplace=True)
# dfA = dfA.iloc[:, : 29]
dfA = dfA.add_prefix('Away ')

In [8]:
dfA.head(10)

,Away Team,Away CF,Away CA,Away CF%,Away FF,Away FA,Away FF%,Away SF,Away SA,Away SF%,...,Away LDCA,Away LDCF%,Away LDSF,Away LDSA,Away LDSF%,Away LDGF,Away LDGA,Away LDGF%,Away LDSH%,Away LDSV%
0,Pittsburgh Penguins,58,55,51.33,47,40,54.02,35,28,55.56,...,39,44.29,9,12,42.86,1,0,100.0,11.11,100.00
1,Seattle Kraken,63,58,52.07,44,47,48.35,31,30,50.82,...,25,54.55,15,15,50.00,1,0,100.0,6.67,100.00
2,Montreal Canadiens,55,64,46.22,41,51,44.57,32,30,51.61,...,20,53.49,11,8,57.89,0,0,NaN,0.00,100.00
3,New York Rangers,53,48,52.48,38,40,48.72,24,27,47.06,...,16,58.97,10,9,52.63,0,1,0.0,0.00,88.89
4,Chicago Blackhawks,61,70,46.56,44,49,47.31,34,36,48.57,...,20,62.26,19,13,59.38,1,1,50.0,5.26,92.31
5,Vancouver Canucks,61,62,49.59,44,45,49.44,38,34,52.78,...,21,58.82,16,10,61.54,2,0,100.0,12.50,100.00
6,Winnipeg Jets,67,39,63.21,48,33,59.26,34,22,60.71,...,16,66.67,14,6,70.00,1,0,100.0,7.14,100.00
7,Montreal Canadiens,54,58,48.21,38,49,43.68,31,36,46.27,...,33,50.00,19,17,52.78,0,2,0.0,0.00,88.24
8,Toronto Maple Leafs,75,68,52.45,63,53,54.31,48,36,57.14,...,16,61.90,16,10,61.54,1,0,100.0,6.25,100.00
9,Tampa Bay Lightning,69,45,60.53,57,35,61.96,48,26,64.86,...,23,58.18,13,8,61.90,0,0,NaN,0.00,100.00


In [9]:
# Merge home and array games to obtain matchup table
df = pd.merge(dfH, dfA, left_index=True, right_index=True)
df.dropna(inplace=True)
df.columns

Index(['Home Team', 'Home CF', 'Home CA', 'Home CF%', 'Home FF', 'Home FA',
       'Home FF%', 'Home SF', 'Home SA', 'Home SF%',
       ...
       'Away LDCA', 'Away LDCF%', 'Away LDSF', 'Away LDSA', 'Away LDSF%',
       'Away LDGF', 'Away LDGA', 'Away LDGF%', 'Away LDSH%', 'Away LDSV%'],
      dtype='object', length=102)

In [10]:
df["Result"] = df["Home GF"]-df["Home GA"]
df.drop(columns=["Home GF", "Home GA", "Away GF", "Away GA"], inplace=True)

In [11]:
df.columns

Index(['Home Team', 'Home CF', 'Home CA', 'Home CF%', 'Home FF', 'Home FA',
       'Home FF%', 'Home SF', 'Home SA', 'Home SF%', 'Home xGF', 'Home xGA',
       'Home xGF%', 'Home SCF', 'Home SCA', 'Home SCF%', 'Home HDCF',
       'Home HDCA', 'Home HDCF%', 'Home HDSF', 'Home HDSA', 'Home HDSF%',
       'Home HDGF', 'Home HDGA', 'Home HDGF%', 'Home HDSH%', 'Home HDSV%',
       'Home MDCF', 'Home MDCA', 'Home MDCF%', 'Home MDSF', 'Home MDSA',
       'Home MDSF%', 'Home MDGF', 'Home MDGA', 'Home MDGF%', 'Home MDSH%',
       'Home MDSV%', 'Home LDCF', 'Home LDCA', 'Home LDCF%', 'Home LDSF',
       'Home LDSA', 'Home LDSF%', 'Home LDGF', 'Home LDGA', 'Home LDGF%',
       'Home LDSH%', 'Home LDSV%', 'Away Team', 'Away CF', 'Away CA',
       'Away CF%', 'Away FF', 'Away FA', 'Away FF%', 'Away SF', 'Away SA',
       'Away SF%', 'Away xGF', 'Away xGA', 'Away xGF%', 'Away SCF', 'Away SCA',
       'Away SCF%', 'Away HDCF', 'Away HDCA', 'Away HDCF%', 'Away HDSF',
       'Away HDSA', 'Away HDSF%', 

In [12]:
X = df.drop(["Result", "Home Team", "Away Team"], axis=1)
y = df['Result']

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

model = RandomForestRegressor(random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [16]:
feature_importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)


In [17]:
top_features = feature_importance_df.head(6)
print("Top 6 features:")
print(top_features)

Top 10 features:
       Feature  Importance
23  Home HDGF%    0.195827
71  Away HDGF%    0.194528
82  Away MDGF%    0.136875
34  Home MDGF%    0.114774
45  Home LDGF%    0.084642
93  Away LDGF%    0.081704
81   Away MDGA    0.008789
80   Away MDGF    0.008118
21   Home HDGF    0.007893
32   Home MDGF    0.007611


In [18]:
from sklearn.metrics import mean_squared_error
y_train_pred = model.predict(X_train_scaled)
train_mse = mean_squared_error(y_train, y_train_pred)
print("Training Mean Squared Error:", train_mse)

y_test_pred = model.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test Mean Squared Error:", test_mse)

Training Mean Squared Error: 0.09976431506849315
Test Mean Squared Error: 0.7179131506849314


In [19]:
from sklearn.model_selection import cross_val_score

num_folds = 5

mse_scores = cross_val_score(model, X, y, cv=num_folds, scoring='neg_mean_squared_error')

mse_scores = -mse_scores

print("Cross-Validation Mean Squared Error Scores:")
print(mse_scores)

mean_mse = mse_scores.mean()
print("Mean MSE:", mean_mse)

Cross-Validation Mean Squared Error Scores:
[0.70847123 0.77112658 0.59822877 0.84210795 0.70494548]
Mean MSE: 0.724976


High errors. Look for methods of improving the model.

In [20]:
from datetime import date, timedelta

# we will use the last 30 days as our data range
today = date.today()
print("Today's date:", today)
start = today - timedelta(days=30)
print("30 days ago:", start)

Today's date: 2024-03-01
30 days ago: 2024-01-31


In [21]:
url = f"https://www.naturalstattrick.com/teamtable.php?fromseason=20232024&thruseason=20232024&stype=2&sit=5v5&score=all&rate=n&team=all&loc=B&gpf=410&fd={start}&td={today}"
req = requests.get(url)
req.status_code

200

In [22]:
df2 = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]

In [23]:
from get_todays_games import getGames

matchups = getGames()
matchups


[('Golden Knights', 'Bruins'),
 ('Canadiens', 'Panthers'),
 ('Hurricanes', 'Blue Jackets'),
 ('Islanders', 'Red Wings'),
 ('Coyotes', 'Maple Leafs'),
 ('Sabres', 'Lightning'),
 ('Jets', 'Stars'),
 ('Wild', 'Predators'),
 ('Avalanche', 'Blackhawks'),
 ('Penguins', 'Kraken'),
 ('Kings', 'Canucks'),
 ('Ducks', 'Sharks')]

In [24]:
res = pd.DataFrame()

for away, home in matchups:
    home_df = df2[df2["Team"].str.contains(home)]
    home_df = home_df.add_prefix('Home ')
    away_df = df2[df2["Team"].str.contains(away)]
    away_df = away_df.add_prefix('Away ')
    home_df = home_df.reset_index(drop=True)
    away_df = away_df.reset_index(drop=True)
    matchup_df = pd.merge(home_df, away_df, left_index=True, right_index=True)
    res = pd.concat([res, matchup_df], ignore_index=True)
    
res.head(10)

,Home Team,Home GP,Home TOI,Home W,Home L,Home OTL,Home ROW,Home Points,Home Point %,Home CF,...,Away LDSA,Away LDSF%,Away LDGF,Away LDGA,Away LDGF%,Away LDSH%,Away LDSV%,Away SH%,Away SV%,Away PDO
0,Boston Bruins,12,586:12,4,3,5,3,13,0.542,546,...,110,45.00,4,2,66.67,4.44,98.18,10.68,90.11,1.008
1,Florida Panthers,11,532:18,9,2,0,8,18,0.818,597,...,107,51.14,5,4,55.56,4.46,96.26,9.76,90.23,1.000
2,Columbus Blue Jackets,9,431:59,3,6,0,3,6,0.333,462,...,117,55.85,3,1,75.00,2.03,99.15,8.01,94.92,1.029
3,Detroit Red Wings,11,549:53,7,3,1,7,15,0.682,538,...,108,51.35,3,6,33.33,2.63,94.44,8.70,91.86,1.006
4,Toronto Maple Leafs,12,609:35,9,3,0,9,18,0.750,617,...,113,48.87,5,2,71.43,4.63,98.23,8.30,88.42,0.967
5,Tampa Bay Lightning,12,576:41,5,6,1,4,11,0.458,563,...,119,49.15,5,2,71.43,4.35,98.32,6.27,94.04,1.003
6,Dallas Stars,13,639:57,6,4,3,6,15,0.577,664,...,112,55.91,4,1,80.00,2.82,99.11,5.71,92.57,0.983
7,Nashville Predators,11,523:28,8,3,0,8,16,0.727,564,...,108,46.53,3,8,27.27,3.19,92.59,9.36,89.57,0.989
8,Chicago Blackhawks,10,512:08,1,6,3,1,5,0.250,414,...,145,49.48,5,5,50.00,3.52,96.55,8.03,92.63,1.007
9,Seattle Kraken,9,426:07,5,3,1,3,11,0.611,436,...,109,53.62,5,2,71.43,3.97,98.17,7.75,92.16,0.999


In [25]:
common_cols = [col for col in res.columns.intersection(df.columns)]
matchups_df = res[common_cols]
final_df = matchups_df.drop(["Home Team", "Away Team"], axis=1)
final_df = final_df.values

In [26]:
predictions = model.predict(final_df)
predictions_df = pd.DataFrame(predictions, columns=["Prediction"])
predictions_df = pd.concat([matchups_df[["Home Team", "Away Team"]], predictions_df], axis=1)
predictions_df

,Home Team,Away Team,Prediction
0,Boston Bruins,Vegas Golden Knights,0.36
1,Florida Panthers,Montreal Canadiens,0.42
2,Columbus Blue Jackets,Carolina Hurricanes,0.42
3,Detroit Red Wings,New York Islanders,0.45
4,Toronto Maple Leafs,Arizona Coyotes,0.36
5,Tampa Bay Lightning,Buffalo Sabres,0.20
6,Dallas Stars,Winnipeg Jets,0.43
7,Nashville Predators,Minnesota Wild,0.38
8,Chicago Blackhawks,Colorado Avalanche,-0.02
9,Seattle Kraken,Pittsburgh Penguins,0.40


In [27]:
f = open("predictions.csv", 'w')

for col in predictions_df.columns.values:
    f.write(col + ",")

f.write("\n")

i = 0
for col in predictions_df.values:
    for row in col:
        f.write(str(row) + ",")
    if i % 2 == 0:
        f.write("\n")
    else:
        f.write("\n\n")
    i += 1

f.close()